
<h1> Lab 3: Reducing Crime</h1>

<h3> Lab_3_Tavares_Iriyama_Akella </h3>

## Introduction

Our team has been hired to understand the determinants of crime to provide policy suggessions to a political party. We have a dataset of crime statistics for a selection of counties in North Carolina. The dataset includes income, demographic, crime statistics and effectiveness of police and judicial bodies. We believe that the determinants of crime should not have changed since 1987 and conclustions and inferences drawn from the dataset could be applied today. 

### Research Question

***How can crime rate be reduced and where can we focus efforts in order to maximize impact?***

In [ ]:
## Installing Packages
suppressWarnings(install.packages("corrplot", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("stargazer", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("car", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("math", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("nbextension", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("ggplot2", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("lmtest", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("sandwich", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("varhandle", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("purrr", repos = "http://cran.us.r-project.org"))
suppressWarnings(install.packages("tidyr", repos = "http://cran.us.r-project.org"))

In [ ]:
## Importing Libraries
suppressWarnings(library(car))
suppressWarnings(library(plyr))
suppressWarnings(library(corrplot))
suppressWarnings(library(stargazer))
suppressWarnings(library(ggplot2))
suppressWarnings(library(lmtest))
suppressWarnings(library(sandwich))
suppressWarnings(library(varhandle))
suppressWarnings(library(purrr))
suppressWarnings(library(tidyr))

### EDA

In [ ]:
A = read.csv("crime_v2.csv")

In [ ]:
dim(A)

In [ ]:
colnames(A)

In [ ]:
summary(A)

**Cleaning up our model to remove NA entries**

In [ ]:
B <- na.omit(A)
dim(B)

**Checking for duplicate entries and retaining only unique values**

In [ ]:
B[duplicated(B),]

In [ ]:
C <- unique(B)
dim(C)

**Viewing the structure of the dataset**

In [ ]:
str(C)

**Correcting 'prbconv' class from Factor to Numeric**

In [ ]:
C$prbconv <- as.numeric(as.character(C$prbconv))
class(C$prbconv)

**Checking the summary of prbconv again**

In [ ]:
summary(C$prbconv)

### Viewing the histogram matrix for variables

In [ ]:
library('tidyverse')
C %>%
  select (-c(county, year)) %>%
  keep(is.numeric) %>% 
  gather() %>% 
  ggplot(aes(value)) +
    facet_wrap(~ key, scales = "free") +
    geom_histogram() + 
    theme (axis.text.x = element_text(angle=45)) + 
    labs(title = "Histogram of Variables")


**prbarr and prbconv - probability values greater than 1**

From the summary and histogram, we noticed probability of values greater than 1. We want to check the observations in order to determine if they should be taken out of our dataset.

In [ ]:
#prbarr
C[C$prbarr > 1.00,]

paste('Number of observations with probability greater than 1 :', count(C[C$prbarr > 1.00,]))

In [ ]:
#prbconv
C[C$prbconv > 1.00,]

paste('Number of observations with probability greater than 1 :', count(C[C$prbconv > 1.00,]))

- The value for prbarr may be an error, however since we only have one value which is close to 1, we have decided not remove it from our dataset.

- We have 10 data points greater than 1 under prbconv. After further deliberation, we have decided that it might be possible to receive multiple convictions for the same crime. We have decided not to exclude these data points.

### Transformations

1) When reviewing the histogram matrix, we noticed a skew to the right for the following variables. We have decided to transform them by taking their log value to expose linearity.

    - crmrte
    - density
    - taxpc
    - prbarr
    - prbconv
    - pctmin80
    
2) In order to work with wages, we decided to first taking the average of all our wage variables to consolidate them into a single variable (avg_wage). We will be taking the log of wages since we want to measure "percent" changes in wages.
    - avg_wage

New variables have been created to capture the log value.

In [ ]:
# Making log of each variables in data C
C$crmrte_log <- log(C$crmrte)

C$density_log <- log(C$density)
C$taxpc_log <- log(C$taxpc)

C$wcon_log <- log(C$wcon)
C$wtuc_log <- log(C$wtuc)
C$wtrd_log <- log(C$wtrd)
C$wfir_log <- log(C$wfir)
C$wser_log <- log(C$wser)
C$wmfg_log <- log(C$wmfg)
C$wfed_log <- log(C$wfed)
C$wsta_log <- log(C$wsta)
C$wloc_log <- log(C$wloc)

C$prbarr_log <-log(C$prbarr)
C$prbconv_log <-log(C$prbconv)
C$prbpris_log <- log(C$prbpris)
C$mix_log <- log(C$mix)
C$pctmin80_log <- log(C$pctmin80)

# Making total_wage and average_wage  
C$total_wage <- C$wcon + C$wtuc + C$wtrd + C$wfir + C$wser + C$wmfg + C$wfed + C$wsta + C$wloc
C$avg_wage <- C$total_wage/9

# Taking log for total_wage and average_wage
C$total_wage_log <- log(C$total_wage)
C$avg_wage_log <- log(C$avg_wage)

In [ ]:
C[c("crmrte_log", "density_log", "taxpc_log", "avg_wage_log",
   "prbarr_log", "prbconv_log", "prbconv_log", "pctmin80_log")] %>%
  keep(is.numeric) %>% 
  gather() %>% 
  ggplot(aes(value)) +
    facet_wrap(~ key, scales = "free") +
    geom_histogram() +
    theme (axis.text.x = element_text(angle=45)) + 
    labs(title = "Histogram of Variables")

**We have noticed an improvement in the skew for certain variables.**

### Checking correlation among variables

In [ ]:
corr_matrix1 <- cor(C[c('crmrte_log', 'prbarr_log', 'prbconv_log', 'prbpris_log', 'avgsen', 'polpc', 'density_log',
                          'taxpc_log', 'west', 'central', 'urban', 'pctmin80_log', 'mix_log', 'pctymle', 'avg_wage_log')])

In [ ]:
corrplot(corr_matrix1, tl.col = "black", method = "circle", type = "upper", tl.srt = 45)

### We analyzed the correlation matrix and found the following interesting correlations:

- Strong correlation between **density** and **crime rate**: we will further analyze **density** variable;
- Moderate correlation between **probability of arrest** and **crime rate**: we wil further analyze **prbarr** variable;
- Moderate correlation between **probability of conviction** and **crime rate**: we wil further analyze **prbconv** variable;
- Moderate correlation between **pctmin80** and **crime rate**: we will further analyze **pctmin80** variable;
- Strong correlation between **urban** and **crime rate**: we are using density as **urban** has a strong correlation with this variable;
- Strong correlation between **tax per capita** and **west** variables.

## Evaluating CLM1 and CLM2:

Sample size is greater than 30 so we can conduct linear regresssion model if the assumptions are met. We will proceed to check the assumptions as follows.

- CLM1: Linearity

We will be assuming a linear relationship between the OLS estimates and the outcome variable, crime rate. 

- CLM2: Random sampling (iid)

Observations were selected randomly, one observation per county. Hence, we can assume that data points are independent and random.
We have verified that we only have one observation for selected counties below:

In [ ]:
table(C$county)

### MODEL 1

In Model 1, we plan to include density due to its high correlation with crime rate.

Density might not only have high statistical significance in our model but also high practical significance. We believe the political party must focus in areas where it can create the maximum impact. We would like to explore this further in model 2.

### Independent Variable - Population Density

In [ ]:
C %>%
    ggplot(aes(density_log)) +
    geom_histogram() + 
    labs(title = "Histogram of Density")

In [ ]:
paste("correlation of crime rate and population density is :", cor(C$crmrte_log, C$density_log))
plot(jitter(C$crmrte_log) ~ jitter(C$density_log), xlab = "Density", ylab = "Crime Rate", main = "Density versus Crime Rate")
(model1 = lm(C$crmrte_log ~ C$density_log, data=C))
abline(model1)

#### Since we have a single point at the far left corner, we would like to check for outliers using Cook's distance prior to validating the CLM assumptions. 

In [ ]:
plot(model1, which=5)

**We have one outlier that is influencing the model. Viewing the outlier.**

In [ ]:
C$county[79]

In [ ]:
C[C$county == 173,]
summary(C$density)

**The density of county 173 is significantly low. We are removing this data point.**

In [ ]:
C_2 <- subset(C, C$county != 173)

**Updating the model**

In [ ]:
paste("correlation of crime rate and population density is :", cor(C_2$crmrte_log, C_2$density_log))
plot(jitter(C_2$crmrte_log) ~ jitter(C_2$density_log), xlab = "Density", ylab = "Crime Rate", main = "Density versus Crime Rate")
(model1 = lm(C_2$crmrte_log ~ C_2$density_log, data=C_2))
abline(model1)

### CLM Assumptions

In [ ]:
plot(model1)

Also checking the histogram of residuals to check if errors are normally distributed.

In [ ]:
hist(model1$residuals, breaks=20, main="Residuals from Linear Model1", col="lightblue")

### Evaluating CLM Assumptions

- CLM3: No Perfect Multicollinearity

Since we only have one independent variable, multicollinearity does not apply here.

- CLM4: Zero Conditional Mean

The residuals vs fitted plot displays a random scatter also represented by a flat band, suggesting that residuals contain no useful information. This also ensures that population actually follows a linear model.

- CLM5: Homoskedasticity

We have reviewed the scale location plot and have noticed a slight skew to the left. We will be using coeftest which is robust in dealing with heteroskedasticity.

- CLM6: Errors are Normally Distributed

The qqplot and histogram of residuals indicates no serious departure from normality.

In [ ]:
## Using coeftest
coeftest(model1, vcov = vcovHC)

In [ ]:
(se.model1 = sqrt(diag(vcovHC(model1))))

In [ ]:
stargazer(model1, type = "text", omit.stat = "f",
          se = list(se.model1),
          star.cutoffs = c(0.05, 0.01, 0.001))

### MODEL 1: NOTES AND CONCLUSION

Density log is a highly statistically significant factor (p-value < 0.001) that influences crime rate. 

For every 1% increase in density, the crime rate is increased about 0.49%. The adjusted $R^2$ is 0.46 implying that 46% of the variations in crime rate can be explained by density variables. The other 54% is due to other factors we will be exploring in model 2.

Finally, we conclude that **population density** is a very important determinant of **crime rate**. Interpreting these results in terms of our research question, **density** can help us with defining where efforts can be focused to maximize the impact of our policy suggestions: we can recommend prioritizing specific actions in counties where density is higher.

## MODEL 2

**Introduction**

In Model 2, we plan to include other variables that were initially identified as potential covariates during EDA. They were initially selected due to their higher correlation with crime rate. The idea here is to improve the model's predictive power while maintaining high statistical significance.

We selected as potential covariates the following variables: **prbarr_log** (Log of Probability of Arrest), **prbconv_log** (Log of Probability of Conviction), **pctmin80_log** (Log of Percent of Minorities) and **wage-related variables**, which are described next.

#### Testing wage-related variables

- To work with **wages**, we decided to first taking the average of all our wage variables to consolidate them into a single variable **(avg_wage)**, in order to avoid reducing our AIC score due to the additional of multiple variables. We then took the log of this average wage variable, as we would like to measure how a percent change in wages affect crime rate.

- When running Model 2 with the selected covariates, we noticed that the log of average of wages **avg_wage_log** variable did not contribute much to the model's predictive power. Also, the variable had low statistical significance per our stargazer table, so we decided to remove this variable from our final Model 2. This happened because there is some collinearity (.57 correlation, per plot below) between the **average wage** variable and the **density** variable (urban areas are more dense in terms of population and tends to drive higher wages): once we added the **density** variable, our **average wage** variable lost statistical significance.

In [ ]:
paste("Correlation of Density and Average Wages is :", cor(C_2$density_log, C_2$avg_wage_log))
#plot(jitter(C_2$density_log) ~ jitter(C_2$avg_wage), data=C_2, xlab = "Density", ylab = "Average Wage", main = "Density versus Avg. Wage")

# Add the regression line
ggplot(C_2, aes(x=density_log, y=avg_wage)) + 
  geom_point()+
  geom_smooth(method=lm)

In [ ]:
cor.test(C_2$density_log, C_2$avg_wage)

### Testing for Multicollinearity among independent variables:

In [ ]:
# Car plot of 4 variables in Model 2
scatterplotMatrix(~C_2$crmrte_log+C_2$density_log+ C_2$prbarr_log + C_2$prbconv_log + C_2$prbconv_log + C_2$pctmin80_log, 
                  diagonal = "histogram")

In [ ]:
#Find correlation matrix
max_corr <- cor(C_2[,c('density_log', 'prbarr_log', 'prbconv_log', 'pctmin80_log')])
max_corr

diag(max_corr) <- 0
paste('Maximum correlation among independent variables is ', max(abs(max_corr)))

**Based on the initial review, we did not notice perfect multicollinearity**

In [ ]:
(model2 = lm(C_2$crmrte_log ~ C_2$density_log + C_2$prbarr_log + C_2$prbconv_log + C_2$pctmin80_log, data=C_2))
summary(model2)$r.square

In [ ]:
## Checking for multicollinearity in model2
vif(model2)
vif(model2) > 4

Since all the variance inflation factors (vif) are less than 4, there exists no multicollinearity in the model.

### CLM Assumptions

In [ ]:
plot(model2)

In [ ]:
hist(model2$residuals, breaks=20, main="Residuals from Linear Model2", col="lightblue")

### Evaluating CLM Assumptions

- CLM3: No Perfect Multicollinearity

We reviewed the correlation plots among our selected covariates and did not identify any perfect correlation among them.

- CLM4: Zero Conditional Mean

We've analyzed Residuals x Fitted plot and did not identify violations of the zero conditional mean. We noticed some variation on the left size, caused by one of the data points. We interpret that this is related to the lack of other datapoints in the same region.

- CLM5: Homoskedasticity

We've also used both the Residuals x Fitted and Scale-Location plots to check for signs of heteroskedasticity. We identified some heteroskedasticity in the plots (band of datapoints reducing from left to right). To mitigate that, we are using White standard errors, which are robust to heteroskedasticity.

- CLM6: Errors are Normally Distributed

We used the Q-Q plot and also a histogram of the residuals and confirmed they are normally distributed.

In [ ]:
## coeftest
coeftest(model2, vcov = vcovHC)

(se.model2 = sqrt(diag(vcovHC(model2))))

stargazer(model1, model2, type = "text", omit.stat = "f",
          se = list(se.model1, se.model2),
          star.cutoffs = c(0.05, 0.01, 0.001))

In [ ]:
paste("AIC score of model1", AIC(model1))
paste("AIC score of model2", AIC(model2))

### MODEL 2: NOTES AND CONCLUSION

#### Notes on Model Building
- We updated model 1 by adding more variables to the model

- We began by using variables identified during EDA as potential good covariates to help improve our model's both predictive power and statistical significance. We identified the following variables: **Probability of Arrest (prbarr)**, **Probability of Conviction (prbconv)**, **Percentage of Minority in 1980 (pctmin80)**.

- We checked for outliers by using Cook's distance plot and did not identify any outliers.


#### Conclusion

Based on Model 2, we conclude that **probability of arrest**, **probability of conviction** and **percent of minorities** are also very important determinants of **crime rate**.

By interpreting our coefficients, we estimate that:

- For every 1% decrease in density, we have a 0.35% reduction in crime rate assuming all other factors stay the same;
- For every 1% increase in probability of arrest, we have a 0.39% reduction in crime rate assuming all other factors stay the same;
- For every 1% increase in probability of conviction, we have a 0.35% reduction in crime rate assuming all other factors stay the same;
- For every 1% decrease in percent of minorities, we have a 0.23% reduction in crime rate assuming all other factors stay the same.

All covariates have very low White standard errors in comparison to their coefficients, confirming the covariates are highly statistically significant. We can also confirm that by looking at their respective p-values in Stargazer plot: all variables are highly statistically significant, at the 99.9% level.

In terms of predictive power, Model 2 can explain 74.4% of observed variation in crime rate based on the selected independent variables. We can also see that Model 2 had great improvement in its predictive power in comparison to Model 1 by comparing their AIC scores: 30.53 vs 93.73 respectively. This confirms Model 2 is a great model to predict crime rate based on the selelected independent variables.

This model is also highly practically significant, as it can be very useful for deciding the policies the political party should endorse and adopt in order to reduce crime rate. Per our coefficient analysis, we see many different opportunities for this, as even some small percentual changes in each independent variable have a strong impact in crime rate.

Finally, interpreting these results in terms of our research question, we can suggest policies that help increase both **probability of arrest** and **probability of conviction**, which will reduce **crime rate**. Also, we plan on recommending policies tailored to the specific needs of minorities, based on our interpretation of **percent of minorities** variable.

We will be discussing the suggested polices further in our conclusion.

## MODEL 3

#### Introduction

In Model 3, we plan to include any other variables that may help us predict **crime rate**. The main idea here is to demonstrate the robustness of our results to model specification. Also, it is an opportunity to check if there are any important variables that were left out of Model 2 and should be included.

We added to the model the following variables: **prbpris_log (Log of Probability of Prison)**, **avgsen_log (Log of Average of Sentence)**, **taxpc_log (Log of Tax per Capita)**, **mix_log (Log of offense mix)** and **wage-related variables** (this time added individually to the model).

In [ ]:
C_2$prbpris_log <- log(C_2$prbpris)
C_2$avgsen_log <- log(C_2$avgsen)
C_2$taxpc_log <- log(C_2$taxpc)
C_2$mix_log <- log(C_2$mix)
C_2$wcon_log <- log(C_2$wcon)
C_2$wtuc_log <- log(C_2$wtuc)
C_2$wtrd_log <- log(C_2$wtrd)
C_2$wfir_log <- log(C_2$wfir)
C_2$wser_log <- log(C_2$wser)
C_2$wmfg_log <- log(C_2$wmfg)
C_2$wfed_log <- log(C_2$wfed)
C_2$wsta_log <- log(C_2$wsta)
C_2$wloc_log <- log(C_2$wloc)

#Find correlation matrix
max_corr_2 <- cor(C_2[,c('density_log', 'prbarr_log', 'prbconv_log', 'pctmin80_log', 'prbpris_log', 'avgsen_log',
                     'taxpc_log', 'mix_log', 'wcon_log', 'wtuc_log', 'wtrd_log', 'wfir_log', 'wser_log', 'wmfg_log',
                     'wfed_log', 'wsta_log', 'wloc_log')])
#max_corr_2

diag(max_corr_2) <- 0
paste('Max correlation among independent variables is ', max(abs(max_corr_2)))


#Adjusting width and height of all plots in this notebook:
options(repr.plot.width=8, repr.plot.height=8)


corr_matrix3 <- cor(C_2[c('density_log', 'prbarr_log', 'prbconv_log', 'pctmin80_log', 'prbpris_log', 'avgsen_log',
                     'taxpc_log', 'mix_log', 'wcon_log', 'wtuc_log', 'wtrd_log', 'wfir_log', 'wser_log', 'wmfg_log',
                     'wfed_log', 'wsta_log', 'wloc_log')])
corrplot(corr_matrix3, tl.col = "black", method = "circle", type = "upper", tl.srt = 45)

In [ ]:
## Building model3
(model3 = lm(C_2$crmrte_log ~ C_2$density_log + C_2$prbarr_log + C_2$prbconv_log + C_2$pctmin80_log +
             C_2$prbpris_log + C_2$avgsen_log + C_2$taxpc_log + C_2$mix_log + C_2$wcon_log + C_2$wtuc_log +
             C_2$wtrd_log + C_2$wfir_log + C_2$wser_log + C_2$wmfg_log + C_2$wfed_log + C_2$wsta_log +
             C_2$wloc_log, data=C_2))
paste('R square for model 3 is :', summary(model3)$r.square)

### CLM Assumptions

In [ ]:
plot(model3)

### Evaluating CLM Assumptions for Model 3


- MLR3: No Perfect Multicollinearity

We reviewed the correlation plots among all our selected covariates and identified some strong collinearity between some of them, so this assumption does not hold.

- MLR4: Zero Conditional Mean

We've analyzed Residuals x Fitted plot and we now identify a violation of the zero conditional mean assumption.

- MLR5: Homoskedasticity

We also see heteroskedasticity in the model now, as datapoints tend to become closer to each other as you look from left to right in the Scale-Location plot.

- MLR6: Errors are Normally Distributed

Normality was reduced drastically, especially at the extremes sides of the Q-Q plot, so errors are now not clearly normally distributed.

In [ ]:
## coeftest
coeftest(model3, vcov = vcovHC)

(se.model3 = sqrt(diag(vcovHC(model3))))

#install.packages("stargazer")
#library(stargazer)
stargazer(model1, model2, model3, type = "text", omit.stat = "f",
          se = list(se.model1, se.model2, se.model3),
          star.cutoffs = c(0.05, 0.01, 0.001))

paste("AIC score of model1", AIC(model1))
paste("AIC score of model2", AIC(model2))
paste("AIC score of model3", AIC(model3))

### MODEL 3: NOTES AND CONCLUSION

#### Notes on Model Building

- For Model 3, we wanted to add any variables that potentially could improve our Model 2, helping us identifying other variables we should consider. We included **probability of prison sentence**, **average sentence** (as we imagined the higher the probability of prison and the average sentence, the lower the **crime rate** would be. We also included **tax per capita** and **offense mix** as there could also be some correlation with crime rate.

- Finally, here we tested again **wages**, trying different combinations including adding each individual wage variable and then taking their individual logs.


#### Conclusion

Model 3 did not improve from Model 2: although we have some minor improvement in both Adjusted R squared and AIC scores, we not only violated a few of the key MLR assumptions, but also all additional covariates included did not have statistical significance (high p-values and most of them had coefficients smaller than their standard errors).

We can also conclude that we did not leave out of Model 2 any other variables that could help us better explain changes in crime rate and prove that Model 2 is a strong model to be used, both in terms of predictive power and statistical significance.

Finally, interpreting these results in terms of our research question: we theorized that **probability of prison sentence**, **average sentence** or **offense mix** would be correlated with crime rate and we could recommend policies that adjusted laws in order to affect them, reducing crime rate - this turned out not true. We also evaluated wage-related variables with **tax per capita** and each individual **wage variables** to confirm they were not significant. We then confirmed that none of those variables would help with our research question.


## Omitted Variables

1) Regression: $crmrte = \beta_0 + \beta_1 density + u$, omitted: $education\_rating\_of\_county$

$$crmrte = \beta_0 + \beta_1*density + \beta_2*education\_rating\_of\_county + u$$

$$education\_rating\_of\_county = \alpha_0 + \alpha_1*density +u$$

- If density is higher, the education quality might suffer;
- If education rating is higher, crime rate will be lower since that implies an engaged and productive society;

If $\beta_2 < 0$ and $\alpha_1 < 0$ then $OMVB = \beta_2\alpha_1 > 0$ and if $\beta_1 > 0$ then the OLS coefficient on $density$ will be scaled away from zero (more positive) gaining statistical significance.

2) Regression: $crmrte = \beta_0 + \beta_1 density + u$, omitted: $social\_morality$

$$crmrte = \beta_0 + \beta_1*density + \beta_2*social\_morality + u$$

$$social\_morality = \alpha_0 + \alpha_1*density +u$$

- When population density is higher, the social morality is generally lower;
- Wheb social morality is lower, the crime rate is higher

If $\beta_2 < 0$ and $\alpha_1 < 0$ then $OMVB = \beta_2\alpha_1 > 0$ and if $\beta_1 > 0$ then the OLS coefficient on $density$ will be scaled away from zero (more positive) gaining statistical significance.

3) Regression: $crmrte = \beta_0 + \beta_1 density + u$, omitted: $alcohol\_and\_drug\_availability$

$$crmrte = \beta_0 + \beta_1*density + \beta_2*alcohol\_and\_drug\_availability + u$$

$$alcohol\_and\_drug\_availability = \alpha_0 + \alpha_1*density +u$$

- When population density is higher, alcohol and drug availability is higher;
- When alcohol and drug availability is higher, crime rate is higher

If $\beta_2 > 0$ and $\alpha_1 > 0$ then $OMVB = \beta_2\alpha_1 > 0$ and if $\beta_1 > 0$ then the OLS coefficient on $density$ will be scaled away from zero (more positive) gaining statistical significance.

4) Regression: $crmrte = \beta_0 + \beta_1 density + u$, omitted: $housing\_affordability$

$$crmrte = \beta_0 + \beta_1*density + \beta_2*housing\_affordability + u$$

$$housing\_affordability = \alpha_0 + \alpha_1*density +u$$

- When population density is higher, housing affordability is lower;
- When housing affordability is lower, crime rate is higher

If $\beta_2 < 0$ and $\alpha_1 < 0$ then $OMVB = \beta_2\alpha_1 > 0$ and if $\beta_1 > 0$ then the OLS coefficient on $density$ will be scaled away from zero (more positive) gaining statistical significance.

5) Regression: $crmrte = \beta_0 + \beta_1 density + u$, omitted: $\_weather$

$$crmrte = \beta_0 + \beta_1*density + \beta_2*weather + u$$

$$weather = \alpha_0 + \alpha_1*density +u$$

- Areas with good weather conditions generally have a higher density;
- When weather is good and more people can be outside, crime rate increases

If $\beta_2 > 0$ and $\alpha_1 > 0$ then $OMVB = \beta_2\alpha_1 > 0$ and if $\beta_1 > 0$ then the OLS coefficient on $density$ will be scaled away from zero (more positive) gaining statistical significance.

## Final Conclusions

### High-Level Concerns of a Political Campaign

We identified three major concerns that the political party has.

1) What is the key actions required?

2) Which policies can be introduced, both short term and long term while creating the maximum impact?

2) How can we prioritize the use the limited resources?


### Discuss Each Variable in Model 2 and interpret them again

Our task was to examine the data to help the campaign understand the determinants of crime and to generate policy suggestions that are applicable to local government.

After exploring the various variables in the dataset, we concluded that the main factors that affect the crime rate are as follows:

- density (log)

As previously mentioned in Model 1's conclusion, we plan to use the **density** variable as a key variable to help us determine which counties to prioritize actions to reduce crime rate.

The model indicates that increase in density tends to increase crime rate.

- probability of arrest (log) and probability of conviction (log)

We want the political party to convey low tolerance to crime so we want to have policy suggestions around policies that can improve **probability of arrest** and **probability of conviction**

The higher the chance that a criminal will be arrested and convicted, the lower the crime rate.

- percent minority 1980 (log)

We are using **pctmin80** in order to understand the groups of population to focus on when proposing inclusion programs.

The higher the minority population, the higher the crime rate.

In summary, density tells us where, probability of arrest and conviction shows us the action that should be taken against criminals and pctmin80 tells us whom to focus on to provide them equal opportunities.


### Policy Suggestions:

***Short Term Impact:***

- Police can play a huge role in reducing crime. We need to deploy police and mobilize resources intelligently, especially in areas where police doing so can have the maximum impact. We suggest policies that expand the resources available to police:

    - Better equipment and technology to identify and arrest suspects: helicopters, cars and technology such as facial recognition;
    - Better equipment and technology for crime scene processing and evidence analysis (investments in forensic labs);
    - Better interrogation training: new techniques, workshops for new officers with learning from experienced ones;
    - Better legal training for police testimonies at court.

Increasing the resources available to police, can help the police collect better evidence and help prosecuters build a better case. This increases the certainty of punishment (do criminals expect to get caught and face punishment).

- Better distribution of the current police force: Hiring more police officers, reallocating them and focusing resources on higher density areas.

- One of the best ways to prevent crime is to provide more resources and support to minority communities. For policymakers, this means expanding access to community resources and workforce development programs. Offer benefits and provide opportunities for advancement.

- Set up subsidized housing and rental programs in low-medium density areas. This will slightly reduce the burden on high-density counties to provide sufficient community resources. 


***Long Term Impact:***

- We understand that change in density cannot be accomplished over the short term. The political party can focus on mobility of population over long term.

- In order to convey a lower tolerance to crime, we advice the politcal party to review and fix major loopholes in laws, thereby increasing the certainty of punishment.

- Increase opportunities for minority population. This can be accomplished through inclusion policies, increased funding for education and professional training.


